In [1]:
import os

import dask
import dask.dataframe as dd
import pandas as pd

In [2]:
len(os.listdir(os.path.join(os.getcwd(),'Apollo_Files')))

22

In [3]:
df = dd.read_csv(os.path.join(os.getcwd(), 'Apollo_Files', '2019*.csv'))

In [4]:
df

,MATERIAL,PLNT,BATCH,UNRESTRICTED,TRANSIT/TRANSF.,IN QUALITY INSP.,RESTRICTED-USE,BLOCKED,RETURNS,STATUS_DATE
npartitions=21,,,,,,,,,,
,object,object,object,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [5]:
df.columns

Index(['MATERIAL', 'PLNT', 'BATCH', 'UNRESTRICTED', 'TRANSIT/TRANSF.',
       'IN QUALITY INSP.', 'RESTRICTED-USE', 'BLOCKED', 'RETURNS',
       'STATUS_DATE'],
      dtype='object')

In [6]:
df = df.rename(columns={'UNRESTRICTED' : "st1",
                       "TRANSIT/TRANSF." : "st2",
                       "MATERIAL" : 'sku_code',
                       "PLNT" : "sc_node_code",
                       "STATUS_DATE" : "status_date",
                       "BATCH" : "batch"})

In [7]:
df['st3'] = df['IN QUALITY INSP.'].astype(float) + df['RESTRICTED-USE'].astype(float)\
            + df['BLOCKED'].astype(float) + df['RETURNS'].astype(float)

In [8]:
df = df.drop(['IN QUALITY INSP.','RESTRICTED-USE','BLOCKED','RETURNS'],axis=1)

In [9]:
df.head()

,sku_code,sc_node_code,batch,st1,st2,status_date,st3
0,RL10J0AS10AH1,1002,LIAW,25.0,0.0,2019-06-01,0.0
1,RL10J0AS10AH1,1002,LIAW,1209.0,0.0,2019-06-01,0.0
2,RL10J0AS10AH1,ASR1,NaN,0.0,67.0,2019-06-01,0.0
3,RL10J0AS10AH1,GJ01,LIAW,3.0,0.0,2019-06-01,0.0
4,RL10J0AS10AH1,GJ02,LIAW,4.0,0.0,2019-06-01,0.0


In [10]:
df_unpivoted =  df.map_partitions(pd.melt, id_vars=['status_date', 'sku_code','sc_node_code','batch'],
                                  var_name='stock_type', value_name='stock_qty')

In [11]:
df_unpivoted = df_unpivoted.rename(columns={'status_date':'for_date'})

In [12]:
df_unpivoted

,for_date,sku_code,sc_node_code,batch,stock_type,stock_qty
npartitions=21,,,,,,
,object,object,object,object,object,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [13]:
forecast_dd = dd.read_csv(os.path.join(os.getcwd(),'Apollo_Files','forecast_june.csv'),
                          dtype={'forecast_qty':float}
                         )

In [14]:
forecast_dd

,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
npartitions=1,,,,,,,,,,
,int64,int64,int64,int64,int64,object,object,object,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [15]:
dbDetails = {
    "driver": "postgresql",
    "username": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "alpha_emami"
}

In [16]:
uri = '%(driver)s://%(username)s:%(password)s@%(host)s:%(port)s/%(database)s' % dbDetails

In [17]:
calendar_dd = dd.read_sql_table('calendar_master',
                                  uri,
                                    schema='bi',
                                  index_col='for_date',
                                  divisions=['2019-06-01','2019-06-07','2019-06-14','2019-06-21','2019-06-30'],
                               ).reset_index()

In [18]:
calendar_dd

,for_date,for_month,for_year,for_day,for_day_reverse,days_in_month,week_in_month,days_week_in_month,week_in_year,days_week_in_year,day_in_week,fiscal_year_1,fiscal_year_2,quarter,fiscal_month,for_month_2,month_year,full_month,attribute_1,attribute_2,attribute_3,week_opp,days_in_week_opp,in_scope
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,
,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,int64,int64,object,object,object,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [19]:
products_dd = forecast_dd[['sku_code','sc_node_code','sales_channel_code','for_month','for_year']].drop_duplicates()

In [20]:
products_dd

,sku_code,sc_node_code,sales_channel_code,for_month,for_year
npartitions=1,,,,,
,object,object,object,int64,int64
,...,...,...,...,...


In [21]:
products_dd = dd.merge(calendar_dd[['for_date','for_month','for_year','days_in_month']], 
         products_dd,
         on=['for_month', 'for_year'])

In [22]:
products_dd

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code
npartitions=4,,,,,,,
,object,int64,int64,int64,object,object,object
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [23]:
forecast_dd

,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
npartitions=1,,,,,,,,,,
,int64,int64,int64,int64,int64,object,object,object,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [25]:
products_dd = dd.merge(products_dd, 
         forecast_dd,
         on=['for_month', 'for_year', 'sku_code', 'sc_node_code','sales_channel_code'],
            how='left')

In [26]:
products_dd['forecast_dly'] = products_dd['forecast_qty']/products_dd['days_in_month']

In [27]:
final_df = products_dd.merge(df_unpivoted,
                 on=['sku_code', 'sc_node_code','for_date'],
                 how='left')

In [28]:
import dask

In [29]:
from dask.distributed import Client

In [30]:
# By default this sets up 1 worker per core
client = Client()
client.cluster

In [31]:
client

Client Scheduler: tcp://127.0.0.1:60749,Cluster Workers: 4 Cores: 8 Memory: 17.03 GB


In [32]:
%%time
forecast_dd.compute().head()

Wall time: 4.2 s


,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
0,1,6,2019,6,2019,RL10J0AS10AH1,1002,REP,1946.0,4378500.0
1,1,6,2019,6,2019,RL10J0AS10AH1,AP01,REP,300.0,675000.0
2,1,6,2019,6,2019,RL10J0AS10AH1,AP02,REP,15.0,33750.0
3,1,6,2019,6,2019,RL10J0AS10AH1,AP03,REP,30.0,67500.0
4,1,6,2019,6,2019,RL10J0AS10AH1,AP04,REP,200.0,450000.0


In [33]:
%%time
df1 = final_df.compute()

Wall time: 1min 2s


In [34]:
df1.head()

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code,source_tag,in_month,in_year,forecast_qty,forecast_value,forecast_dly,batch,stock_type,stock_qty
0,2019-06-07,6,2019,30,RL10J0AS10AH1,OR02,REP,1,6,2019,2.0,4500.0,0.066667,NaN,NaN,NaN
1,2019-06-07,6,2019,30,RL10J0AS10AH1,RJ01,REP,1,6,2019,10.0,22500.0,0.333333,NaN,NaN,NaN
2,2019-06-07,6,2019,30,RL10J0AS10AH1,RJ09,REP,1,6,2019,0.0,0.0,0.000000,NaN,NaN,NaN
3,2019-06-07,6,2019,30,RL10O0AS10AH1,AP05,REP,1,6,2019,30.0,70500.0,1.000000,NaN,NaN,NaN
4,2019-06-07,6,2019,30,RL10O0AS10AH1,BR03,REP,1,6,2019,0.0,0.0,0.000000,NaN,NaN,NaN


In [ ]:
del df1